# Neste Trabalho vamos criar um preditor de preços com regressões lineares.

Ele não tem a intenção de ser um trabalho completo ou um estudo sobre a elasticide-preço, apenas uma introdução que passa por diversos problemas comuns nesse tipo de estudo e algumas possíveis soluções a esses problemas.

O projeto

In [ ]:
!pip install pandas
!pip install numpy
!pip install sklearn
!pip install matplotlib
!pip install scipy
!pip install statsmodels
!pip install keras
!pip install pysal
!pip install geopandas
!pip install mplleaflet

  Running setup.py bdist_wheel for sklearn: started
  Running setup.py bdist_wheel for sklearn: finished with status 'done'
  Stored in directory: C:\Users\ABC\AppData\Local\pip\Cache\wheels\d7\db\a3\1b8041ab0be63b5c96c503df8e757cf205c2848cf9ef55f85e
Successfully built sklearn
  Using cached geopandas-0.2.1-py2.py3-none-any.whl
  Using cached pyproj-1.9.5.1.tar.gz
  Using cached Fiona-1.7.9.post1.tar.gz
  Using cached descartes-1.1.0-py3-none-any.whl
  Using cached Shapely-1.6.0.tar.gz
    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "C:\Users\ABC\AppData\Local\Temp\pip-build-qmu2l_9k\shapely\setup.py", line 80, in <module>
        from shapely._buildcfg import geos_version_string, geos_version, \
      File "C:\Users\ABC\AppData\Local\Temp\pip-build-qmu2l_9k\shapely\shapely\_buildcfg.py", line 200, in <module>
        lgeos = CDLL("geos_c.dll")
      File "C:\Users\ABC\Anaconda\lib\c

Command "python setup.py egg_info" failed with error code 1 in C:\Users\ABC\AppData\Local\Temp\pip-build-qmu2l_9k\shapely\


In [ ]:
# Math, stat and data
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.formula.api as smf

# sklearn for regressions
from sklearn import ensemble
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline

# keras for deep learning
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

# packages for geopatial regressions
import pysal as ps
import geopandas as gpd
import mplleaflet as mpll

# packages for viz
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image

%matplotlib inline

Using TensorFlow backend.


In [ ]:
path = 'C:\\Users\\ABC\\OneDrive\\data\\pricing_predict\\data\\house_sales.csv'
# path = '/Users/sn3fru/OneDrive/data/pricing_predict/data/house_sales.csv'
# path = 'house_sales.csv'

# 1) Analise exploratória

#### Antes de mais nada vamos dar uma boa olhada no comportamento das distribuições cruzadas com a função pairplot do seaborn, com isso consigo ter uma boa ideia de quais variaveis são fortes candidatos a serem tratadas, quais tem alta correlação e quais são boas candidatas para dummies

In [ ]:
df = pd.read_csv(path)

In [ ]:
df.head(10)

In [ ]:
df.describe()

In [ ]:
g = sns.pairplot(df, hue="condition")
# g.savefig('first-pairplot')

### temos mais variaveis do que eu imaginava para fazer isso no olho, vamos ver como é a correlação geral dessas variaveis ...

In [ ]:
corrmat = df.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True)

### Parece que muitas variaveis tem baixa correlação com as demais (pelo menos sem fazer algum tratamento) com isso vamos plotar apenas as 10 variaveis com maiores correlações com o preço.

Estranho notar a latitude e longitude sendo sendo bastante correlacionado, isso só faria sentido se a amostra for toda de uma pequena região onde haveria um gradiente de preços. Se Fosse todo os EUA, esse feito linear se anularia.

Vamos ver.

In [ ]:
k = 10
f, ax = plt.subplots(figsize=(12, 9))
cols = corrmat.nlargest(k, 'price')['price'].index
cm = np.corrcoef(df[cols].values.T)
sns.set(font_scale=1.15)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)

### Checamos que a base não tem valores nulos: (mas tem muitos valores zero onde deveria ser nulo)

In [ ]:
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

In [ ]:
### checamos quais as variaveis são muito assimétricas:
numeric_feats = df.dtypes[df.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = df[numeric_feats].apply(lambda x: stats.skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)

## Uma breve olhada nos dados geograficos fornecidos já nos dizem muito sobre as possibilidades dessa base.

Se tivessemos uma amostra de todo os EUA, não faria muito sentido usar autocorrelação espacial já que os efeitos são dissipados rápidamente, mas como estamos falando de uma area razoavelmente pequena, um único distrito, King County (em Washington), podemos não só usar as autocorrelações espaciais, como podemos usar os dados públicos como o censu americano ( https://www.census.gov ) e até dados de GIS (https://gis-kingcounty.opendata.arcgis.com/) para enriquecer nosso preditor de preços de venda.



In [ ]:
points = np.arange(df.shape[0])
np.random.shuffle(rids)
f, ax = plt.subplots(1, figsize=(9, 9))
df.iloc[points[:500], :].plot(kind='scatter', x='longitude',
                             y='latitude',s=30, linewidth=0, ax=ax);
mpll.display(fig=f,)

### mapa plotado no Tableau Public, os números nas regiões representam a quantidade unidades vendidas e a intensidade das cores é o preço médio das unidades conforme escala abaixo

In [ ]:
Image(filename='C:\\Users\\ABC\\Desktop\\tableau-map.jpg') 

### Por último vamos rodar uma simples regressão nos dados sem tratamento para ver como as variaveis se comportam ceteris paribus, ou seja, o valor dos betas, seus sinais e suas significancias estatisticas.

Tem muita coisa para se analisar na regressão abaixo, mas vamos falar só do genérico e deixar para as próximas com variaveis tratadas.

Temos o um R2 de 0.577 com muita assimetria e curtose elevada indicando residuos mal distribuidos.

In [ ]:
function1 = '''
price ~ num_bed
 + num_bath
 + size_house
 + size_lot
 + num_floors
 + is_waterfront
 + condition
 + size_basement
 + renovation_date
 + avg_size_neighbor_houses
 + avg_size_neighbor_lot
 + zip
'''

model1 = smf.ols(function1, df).fit()
print(model1.summary2())

# 2) Engenharia de Variaveis

A maior, mais díficil e mais importante etapa.

### Embora os dados exibam um comportamento linear, a amostra é bastante heterocedastica.

Analisando o primeiro pairplor, notamos séries 'explosivas', para torna-la homocedastica podemos aplicar diversas técnicas, usaremos o mais simples que é a aplicação de log nas duas variaveis pois além deconseguimos a variância constante, temos uma interpretação de elasticidade (taxa de variação) para os parâmetros.


Abaixo a relação entre Preços e o tamanho da casa que imaginamos que tenha uma forte e positiva correlação.

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = df['size_house'], y = df['price'])
plt.ylabel('Price', fontsize=13)
plt.xlabel('Size', fontsize=13)
plt.show()

#### Descreverei como log-linearizaremos' as principais variaveis com mais detalhes e as demais será feito o procedimento (se necessário) sem explicações

Não necessáriamente precisamos ter distribuições normais para nossas amostras, mas ela ter essa caracteristica permite que façamos analises não só mais eficientes mas principalmente mais robustas já que a maioria dos algoritmos de regressões que usaremos trazem betas significativos para qualquer distribuição apenas com o primeiro momento, mas não somos capazes de fazer testes de hipótese sem o segundo momento.

In [ ]:
sns.distplot(df['price'] , fit=stats.norm);

(mu, sigma) = stats.norm.fit(df['price'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('Price distribution')

fig = plt.figure()
res = stats.probplot(df['price'], plot=plt)
plt.show()

#### E após a aplicação de logaritmo natural na variavel ...

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = np.log(df['size_house']), y = np.log(df['price']))
plt.ylabel('Price', fontsize=13)
plt.xlabel('Size', fontsize=13)
plt.show()


sns.distplot(np.log(df['price']) , fit=stats.norm);

# Get the fitted parameters used by the function
(mu, sigma) = stats.norm.fit(np.log(df['price']))
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('Price distribution')

fig = plt.figure()
res = stats.probplot(np.log(df['price']), plot=plt)
plt.show()

#### A segunda variavel com maior correlação é a média de tamanho das casas da vizinhança.
Uma forma alternatica de transformação essa informação em uma variavel explicativa interessante
é considerar não exatamente o tamanho média das casas do bairro, mas o quanto estamos proximos ou longe
da média das casas do bairro. Pois se a média é grande ou pequena mas nossa casa pode ser ainda maior ou ainda menor não fazendo a comparação relativa. Portanto vamos analisar a variavel, log-lineariza-la e criar uma nova que será uma proporção entre a venda e a sua média. 

Obs: Não faria sentido criar novas variaveis que fossem combinações lineares entre as variaveis do modelo, muitos softwares simplesmente não rodam por não conseguirem inveter a matriz de parametros (X) acusando multicolinearidade perfeita (ou em termos mais economicos, a variavel criada como combinação linear de outras não acrescenta nenhum novo poder explicativo ao modelo), neste casso criaremos um indice percentual para testarmos sua correlação com o preço.

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = df['avg_size_neighbor_houses'], y = df['price'])
plt.ylabel('Price', fontsize=13)
plt.xlabel('neighbor', fontsize=13)
plt.show()

res1 = stats.probplot(df['avg_size_neighbor_houses'], plot=plt)
plt.show()

fig, ax = plt.subplots()
ax.scatter(x = np.log(df['avg_size_neighbor_houses']), y = np.log(df['price']))
plt.ylabel('Price', fontsize=13)
plt.xlabel('neighbor', fontsize=13)
plt.show()

res2 = stats.probplot(np.log(df['avg_size_neighbor_houses']), plot=plt)
plt.show()

#### Embora a aplicação do log resolva facilmente alguns possíveis preblemas de distribuições 'mal comportadas' ele não resolve bem problemas de indices, pois a ideia é de que uma regressão log-log tenha variações constantes, dando a interpretação de 'ao variar nossa endógena em x% a exógena variará em \beta %'

In [ ]:
df2 = df.copy(deep=True)
df2['sizediff_relative_of_neighbor'] = (df2['avg_size_neighbor_houses']-df2['size_house'])/df2['size_house']

fig, ax = plt.subplots()
ax.scatter(x = df2['avg_size_neighbor_houses'], y = df2['size_house'])
plt.ylabel('size', fontsize=13)
plt.xlabel('avg neighbor', fontsize=13)
plt.show()

fig, ax = plt.subplots()
ax.scatter(x = np.log(df2['avg_size_neighbor_houses']), y = np.log(df2['size_house']))
plt.ylabel('log size', fontsize=13)
plt.xlabel('log avg neighbor', fontsize=13)
plt.show()

res0 = stats.probplot(df2['avg_size_neighbor_houses'], plot=plt)
plt.show()

#### Relação entre distancia percentual para a média e o Preço de venda
obs: essa variavel não poderá ser usada nas regressões pois um dos inputs é a variavel exógena, usamos para entender a relação da variavel principal.

In [ ]:
sns.jointplot(df2['sizediff_relative_of_neighbor'], np.log(df2['price']), kind="kde")
plt.ylabel('log Price', fontsize=13)
plt.xlabel('diff neighbor', fontsize=13)
plt.show()

res1 = stats.probplot(df2['sizediff_relative_of_neighbor'], plot=plt)
plt.show()

In [ ]:
df2['log_price'] = np.log(df2['price'])
df2['log_size_house'] = np.log(df2['size_house'])
df2['size_lot']
df2['log_size_lot'] = np.log(df2['size_lot'])
df2['log_neighbor_house'] = np.log(df2['avg_size_neighbor_houses'])
df2['log_neighbor_lot'] = np.log(df2['avg_size_neighbor_lot'])
df2['log_size_basement'] = np.log(df2['size_basement'] + 1)

In [ ]:
mask1 = (df2['year_built'] != 0)
z_valid = df2[mask1]

df2['time_since_construction'] = 50
df2.loc[mask1, 'time_since_construction'] = np.log(2016 - z_valid['year_built'])

In [ ]:
mask2 = (df2['renovation_date'] != 0)
z_valid = df2[mask2]

df2['time_since_renovation'] = 50
df2.loc[mask2, 'time_since_renovation'] = np.log(2016 - z_valid['renovation_date'])

## Variaveis candidatas a Categóricas (classes)

In [ ]:
g = sns.factorplot(x="num_bath", data=df2, kind="count",
                   palette="BuPu", size=6, aspect=1.5)
g.set_xticklabels(step=2)

df2['c_num_bath'] = df2.num_bath.replace({0:0, 0.50:0, 0.75:0,
                                          1:1, 1.25:1, 1.50:1, 1.75:1,
                                          2:2, 2.25:2, 2.50:2, 2.75:2,
                                          3:3, 3.25:3, 3.50:3, 3.75:3,
                                          4:4, 4.25:4, 4.50:4, 4.75:4,
                                          5:5, 5.25:5, 5.50:5, 5.75:5,
                                          6:5, 6.25:5, 6.50:5, 6.75:5,
                                          7:5, 7.25:5, 7.50:5, 7.75:5,
                                          8:5})

g = sns.factorplot(x="c_num_bath", data=df2, kind="count",
                   palette="BuPu", size=6, aspect=1.5)
g.set_xticklabels(step=2)

df2['c_num_partial_bath'] = df2.num_bath.replace({0:0, 0.25:1, 0.50:2, 0.75:3,
                                                  1:0, 1.25:1, 1.50:2, 1.75:3,
                                                  2:0, 2.25:1, 2.50:2, 2.75:3,
                                                  3:0, 3.25:1, 3.50:2, 3.75:3,
                                                  4:0, 4.25:1, 4.50:2, 4.75:3,
                                                  5:0, 5.25:1, 5.50:2, 5.75:3,
                                                  6:0, 6.25:1, 6.50:2, 6.75:3,
                                                  7:0, 7.25:1, 7.50:2, 7.75:3,
                                                  8:0})

g = sns.factorplot(x="c_num_partial_bath", data=df2, kind="count",
                   palette="BuPu", size=6, aspect=1.5)
g.set_xticklabels(step=2)

In [ ]:
g = sns.factorplot(x="num_bed", data=df2, kind="count",
                   palette="BuPu", size=6, aspect=1.5)
g.set_xticklabels(step=2)

df2['c_num_bed'] = df2.num_bed.replace({ 0:1, 1:1, 2:2, 3:3, 4:4, 5:5, 6:6, 7:6, 8:6, 9:6, 10:6, 33:6})

g = sns.factorplot(x="c_num_bed", data=df2, kind="count",
                   palette="BuPu", size=6, aspect=1.5)
g.set_xticklabels(step=2)

In [ ]:
df2['c_num_floors'] = df2.num_floors.replace({ 1:1, 1.5:1, 2:2, 2.5:2, 3:3, 3.5:3})
df2['c_num_partial_floors'] = df2.num_floors.replace({ 1:0, 1.5:1, 2:0, 2.5:1, 3:0, 3.5:1})

## Para evitarmos de criar dummies com o zip code que embora aumente o poder explicativo, deixamos os residuos novamente com o terceiro e quarto momentos ainda mais distantes da gaussiana, vamos criar um segundo dataframe com os valores médios agrupados por zip e usar como variaveis endógenas (aceitando a multicolineariedade ... )

In [ ]:
# spatial
avg_zip = df2.groupby(df2['zip'])[['price', 'size_house', 'size_lot','latitude','longitude','year_built']].median()
avg_zip.rename(columns={'price': 'zip_price',
                        'size_house': 'zip_size_house',
                        'size_lot':'zip_size_lot',
                        'latitude': 'zip_latitude',
                        'longitude': 'zip_longitude',
                        'year_built': 'zip_year_built'}, inplace=True)
avg_zip = avg_zip.reset_index()
df2 = df2.merge(avg_zip, left_on='zip', right_on='zip', how='inner')

In [ ]:
df2['zip_log_price'] = np.log(df2['zip_price'])
df2['zip_log_size_house'] = np.log(df2['zip_size_house'])
df2['zip_log_size_lot'] = np.log(df2['zip_size_lot'])

In [ ]:
df2.head()

## Vamos rodar novamente uma OLS simples para ver como as variaveis passaram a se comportar, seus testes conjuntos, assimetria, curtose e R2.

Como evidenciado abaixo, conseguimos tanto aumentar o poder explicativo do modelo sem causar overfiting, além disso diminuimos muito a assimetria dos residuos e embora a curtose não seja baixa ela é 1/6 da inicial!

In [ ]:
function2 = '''
log_price ~ sizediff_relative_of_neighbor
 + C(c_num_bed)
 + C(c_num_bath)
 + C(c_num_partial_bath)
 + log_size_house
 + log_size_lot
 + C(c_num_floors)
 + C(is_waterfront)
 + C(condition)
 + log_size_basement
 + time_since_renovation
 + time_since_construction
 + log_neighbor_house
 + log_neighbor_lot
 + np.log(zip_size_house)
 + np.log(zip_price)
 + np.log(zip_size_lot)
'''

model2 = smf.ols(function2, df2).fit()
print(model2.summary2())

In [ ]:
df2.rename(columns={'condition': 'c_condition', 'is_waterfront':'c_is_waterfront'}, inplace=True)

df3 = df2[['log_price', 'zip', 'latitude', 'longitude', 'sizediff_relative_of_neighbor',
           'log_size_house', 'log_size_lot', 'log_neighbor_house', 'log_neighbor_lot',
           'log_size_basement', 'time_since_construction', 'time_since_renovation',
           'zip_log_size_house', 'zip_log_size_lot', 'zip_log_price','c_condition','c_num_floors',
           'c_num_partial_floors', 'c_num_bath','c_num_partial_bath','c_num_bed', 'c_is_waterfront']]

## As variaveis com maiores e menores correlaçõoe com Preço.

In [ ]:
corr = df3.corr()
corr.sort_values(["log_price"], ascending = False, inplace = True)
print(corr.log_price)

#### Nesse método vamos transformar nossas variaveis categóricas em dummies para permitir que a variação da variavel tenha efeitos diferentes sobre o preço, além de abrir a possibilidade de iterarmos as dummies com as variaveis continuar para flexibilizar o valor dos parametros, ou seja, torna-los condicionais a ter ou não a dummy diferente da regressão comum que considera o efeito ceteris paribus.

In [ ]:
categoricals = ['c_condition','c_num_floors','c_num_partial_floors',
                'c_num_bath','c_num_partial_bath','c_num_bed', 'c_is_waterfront', 'zip']

def one_hot(df, cols):
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
    return df

df3 = one_hot(df3,categoricals)

df3 = df3.drop(categoricals, axis=1)

# Modelos


Veremos agora alguns modelos bastante simples com suas próprias caracteristicas que podemos testar e combina-los para conseguir tirar o melhor previsor da combinação dos modelos (método de ensemble). 

## Antes de mais nada criamos algumas funções de apoio para medirmos o desempenho desses modelos. 

## Abaixo a função de split que divide nosso dataframe em 2 partes de forma aleatória, uma para treinarmos os modelos e outra para testarmos. É especialmente importante que não usemos os dados de treinamento para testar pois se nosso modelo ficar perfeitamente bom para os dados treinados pode ser que ele não seja genérico o suficiente para ser extrapolado para outros dados, a separação nessa etapa tem esse proposóto

In [ ]:
def dfSplit(df,ratio,y='log_price'):
    train, test = train_test_split(df, test_size = ratio)
    y_train = train[y]
    y_test = test[y]
    x_train = train.ix[:, train.columns != y]
    x_test = test.ix[:, test.columns != y]
    return x_train, y_train, x_test, y_test

## Após criar e ajustar diversas variaveis nas etapas anteriores, ainda assim podemos criar polinomios para que nossa função obtenha a flexibilidade de:



## (1) se ajustar a funções quadraticas e principalmente tornar os parametros marginalmente decrescente, ou seja, tem sua primeira derivada positiva e a segunda negativa e 



## (2) permitir interação entre os efeitos, ou seja, será que o efeito do aumento do tamanho da casa se mantém constante quando outras variaveis variam? Acho que não, e podemos testar essas interações criando polinomios multiplicativos.



obs: Essa inserção de parametros multiplicativos não quebra a constistência e robustez da OLS mesmo ela sendo 'linear', pois os parâmetros ajustados (os betas) continuam sendo lineares!

In [ ]:
def tPoly(df, degree=1):
    polynomial = PolynomialFeatures(degree=degree)
    
    return polynomial.fit_transform(df)

## Métricas de ajustes.


#### Diferente das métricas de classificação (precisão, recall, f1-score, etc) as métricas de regressões são um pouco mais complicadas e trabalharemos com as mais simples, a RMSE que tem seus defeitos mas será útil nesse caso e nosso R2.

In [ ]:
scorer = make_scorer(mean_squared_error, greater_is_better = False)


def testRegs(df, clf, degree=1, ratio=.2, y='log_price', metrics=[]):

    x_train,y_train,x_test,y_test = dfSplit(df,ratio,y='log_price')

    poly_x_train = tPoly(x_train,degree)
    poly_x_test = tPoly(x_test,degree)

    clf.fit(poly_x_train,y_train)
    
    y_hat = clf.predict(poly_x_test)
    
    rmse = np.sqrt(-cross_val_score(clf, poly_x_train, y_train, scoring = scorer, cv = 10))
    
    print("RMSE:", rmse.mean())
    
    y_train_pred = clf.predict(poly_x_train)
    y_test_pred = clf.predict(poly_x_test)

    print('R2: %.2f, Score: %.2f, Parameters: %i' % (r2_score(y_test, y_hat), 
                                                     clf.score(poly_x_train,y_train), 
                                                     clf.coef_.shape[0]))

    plt.scatter(y_train_pred, y_train_pred - y_train, c = "blue", marker = "s", label = "Training data")
    plt.scatter(y_test_pred, y_test_pred - y_test, c = "lightgreen", marker = "s", label = "Validation data")
    plt.xlabel("Predicted values")
    plt.ylabel("Residuals")
    plt.hlines(y = 0, xmin = 10.5, xmax = 13.5, color = "red")
    plt.show()

    plt.scatter(y_train_pred, y_train, c = "blue", marker = "s", label = "Training data")
    plt.scatter(y_test_pred, y_test, c = "lightgreen", marker = "s", label = "Validation data")
    plt.xlabel("Predicted values")
    plt.ylabel("Real values")
    plt.legend(loc = "upper left")
    plt.plot([10.5, 13.5], [10.5, 13.5], c = "red")
    plt.show()

## Regrediremos com OLS os dados sobre diferentes valores polinomias para ter uma noção geral de sobreajuste

In [ ]:
ols_simple = linear_model.LinearRegression()

for i in range(1,3):
    testRegs(df3,ols_simple,i)

## Como sabemos que nossa base de dados tem bastante multicolinearidade entre os parametros, vamos usar regressões robustas a isso para minimizar esses efeitos de overfiting.

Nessas regressões com regularização, basicamente é adicionado viés aos parâmetros para penalizar os efeitos de valores extremos. Ridge usa L2 e Lasso usa L1.

In [ ]:
ridge = linear_model.RidgeCV(alphas = [0.001, 0.01, 0.1, 0.5, 0.75, 1, 1.2, 1.5, 2.5, 5])
testRegs(df3,ridge)


In [ ]:
lasso = linear_model.LassoCV(alphas = [0.0001, 0.0003, 0.0006, 0.001, 0.003,
                                       0.006, 0.01, 0.03, 0.06, 0.1, 0.3, 0.6, 1],
                                       max_iter = 50000, cv = 10)
testRegs(df3,lasso)

In [ ]:
!pip install xgboost
import xgboost as xgb



In [ ]:
ensemble = ensemble.GradientBoostingRegressor(n_estimators = 500, 
                                              max_depth = 5, 
                                              min_samples_split = 2)

x_train, y_train, x_test, y_test = dfSplit(df3,.1)
gbr = ensemble.fit(x_train, y_train)
print(ensemble.score(x_test, y_test))
print(np.sqrt(-cross_val_score(gbr, x_train, y_train, scoring = scorer, cv = 5))).mean()

## Inserindo geolocalização como variaveis autocorrelacionadas endógenas.

In [ ]:
sns.jointplot(x=df.longitude, y=df.latitude, size=8, kind='kde', space=0)

In [ ]:
from sklearn.metrics import mean_squared_error as mse

mses = pd.Series({'OLS': mse(y, m1.predy.flatten()), \
                     'OLS+W': mse(y, m2.predy.flatten()), \
                     'Lag': mse(y, m3.predy_e)
                    })
mses.sort_values()

### Uma possibilidade que está bastante na moda é usar Deep learning (nne com camadas ocultas), eu honestamente acho uma ótima ideia para diversos problemas como imagens, sons e outros problemas não linearizaveis, mas o fato de podermos escrever uma função qualquer  não linearizavel quando sabemos que a função deveria ter parâmetros lineares só abre a possibilidade de overfiting desnecesário.

Abaixo alguns casos.

In [ ]:
def simple_model():
    model = Sequential()
    model.add(Dense(50, input_dim=42, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
estimator = KerasRegressor(build_fn=simple_model, nb_epoch=10000, batch_size=1000, verbose=False)

x_train, y_train, x_test, y_test = dfSplit(df3,.1)

kfold = KFold(n_splits=5)
results = cross_val_score(estimator, x_train.values, y_train, cv=kfold)

In [ ]:
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

In [ ]:
def simple_model_with_hidden():
    model = Sequential()
    model.add(Dense(100, input_dim=42, kernel_initializer='normal', activation='relu'))
    model.add(Dense(25, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

estimator = KerasRegressor(build_fn=simple_model_with_hidden, nb_epoch=10000, batch_size=1000, verbose=False)

x_train, y_train, x_test, y_test = dfSplit(df3,.01)

kfold = KFold(n_splits=5)
results = cross_val_score(estimator, x_train.values, y_train, cv=kfold)

In [ ]:
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))